<a href="https://colab.research.google.com/github/quiidich/3-min-pytorch/blob/master/Burgers_Eq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from random import uniform
from functools import partial

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

iter = 0

In [2]:
def set_seed(seed: int = 42):
    '''
    Seeding the random variables for reproducibility
    '''
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
class BurgersNN(nn.Module):
    def __init__(self,):
        super(BurgersNN, self).__init__()
        # Input layer
        self.linear_in = nn.Linear(2, 20)
        # Output layer
        self.linear_out = nn.Linear(20, 1)
        # Hidden layers
        self.layers = nn.ModuleList(
            [nn.Linear(20, 20) for i in range(8)]
        )
        # Activation function
        self.act = nn.Tanh()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear_in(x)
        x = self.act(x)
        for layer in self.layers:
            x = self.act(layer(x))
        x = self.linear_out(x)
        return x

In [4]:
def derivative(dy: torch.Tensor, x: torch.Tensor, order: int = 1) -> torch.Tensor:
    """
    This function calculates the derivative of the model at x_f
    """
    for i in range(order):
        dy = torch.autograd.grad(
            dy, x, grad_outputs = torch.ones_like(dy), create_graph=True, retain_graph=True
        )[0]
    return dy

In [5]:
def u_function(model: BurgersNN, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    """
    This function evaluates the model on the input x
    """
    model_input = torch.stack((x, t), axis = 1)
    return model(model_input)

In [6]:
def f(model, x_f, t_f):
    u = u_function(model, x_f, t_f)
    u_t = derivative(u, t_f, order=1)
    u_x = derivative(u, x_f, order=1)
    u_xx = derivative(u, x_f, order=2)
    f = u_t + u.T*u_x - (0.01/np.pi)*u_xx
    return f

In [7]:
def loss_function(model: BurgersNN, x_u: torch.Tensor, x_f: torch.Tensor, t_f: torch.Tensor, t_u: torch.Tensor, y_u: torch.Tensor) -> torch.Tensor:
    """
    This function evaluates the physics governing the model on the input x_f
    """
    u = u_function(model, x_f, t_f)
    MSE_f = f(model, x_f, t_f).pow(2).mean()
    MSE_u = (u_function(model, x_u, t_u)-y_u).pow(2).mean()
    return MSE_f + MSE_u

In [8]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)

In [9]:
def closure(model: BurgersNN, optimizer, X_u_train: torch.Tensor, X_f_train:torch.Tensor, Y_u_train: torch.Tensor) -> torch.Tensor:
    """
    In order to use the LBFGS optimizer, we need to define a closure function. This function is called by the optimizer
    and the optimizer contains the inner loop for the optimization and it continues until the tolerance is met.
    """
    x_u = X_u_train[:, 0]
    t_u = X_u_train[:, 1]
    x_f = X_f_train[:, 0]
    t_f = X_f_train[:, 1]
    y_u = Y_u_train
    optimizer.zero_grad()
    loss = loss_function(model, x_u, x_f, t_f, t_u, y_u)
    loss.backward()
    global iter
    iter += 1
    print(f" iteration: {iter}  loss: {loss.item()}")
    return loss

In [10]:
def train(model, X_u_train, X_f_train, u_train):
    # Initialize the optimizer
    optimizer = torch.optim.LBFGS(model.parameters(),
                                    lr=1,
                                    max_iter=50000,
                                    max_eval=50000,
                                    history_size=50,
                                    tolerance_grad=1e-05,
                                    tolerance_change=0.5 * np.finfo(float).eps,
                                    line_search_fn="strong_wolfe")

    # the optimizer.step requires the closure function to be a callable function without inputs
    # therefore we need to define a partial function and pass it to the optimizer
    closure_fn = partial(closure, model, optimizer, X_u_train, X_f_train, u_train)
    optimizer.step(closure_fn)

In [ ]:
if __name__ == "__main__":
    # Set seed for reproducibility
    set_seed(42)

    nu = 0.01 / np.pi         # constant in the diff. equation
    N_u = 100                 # number of data points in the boundaries
    N_f = 10000               # number of collocation points

    # X_u_train: a set of pairs (x, t) located at:
        # x =  1, t = [0,  1]
        # x = -1, t = [0,  1]
        # t =  0, x = [-1, 1]
    x_upper = np.ones((N_u//4, 1), dtype=float)
    x_lower = np.ones((N_u//4, 1), dtype=float) * (-1)
    t_zero = np.zeros((N_u//2, 1), dtype=float)

    t_upper = np.random.rand(N_u//4, 1)
    t_lower = np.random.rand(N_u//4, 1)
    x_zero = (-1) + np.random.rand(N_u//2, 1) * (1 - (-1))

    # stack uppers, lowers and zeros:
    X_upper = np.hstack( (x_upper, t_upper) )
    X_lower = np.hstack( (x_lower, t_lower) )
    X_zero = np.hstack( (x_zero, t_zero) )

    # each one of these three arrays haS 2 columns,
    # now we stack them vertically, the resulting array will also have 2
    # columns and 100 rows:
    X_u_train = np.vstack( (X_upper, X_lower, X_zero) )

    # shuffle X_u_train:
    index = np.arange(0, N_u)
    np.random.shuffle(index)
    X_u_train = X_u_train[index, :]

    # make X_f_train:
    X_f_train = np.zeros((N_f, 2), dtype=float)
    for row in range(N_f):
        x = uniform(-1, 1)  # x range
        t = uniform( 0, 1)  # t range

        X_f_train[row, 0] = x
        X_f_train[row, 1] = t

    # add the boundary points to the collocation points:
    X_f_train = np.vstack( (X_f_train, X_u_train) )

    # make u_train
    u_upper =  np.zeros((N_u//4, 1), dtype=float)
    u_lower =  np.zeros((N_u//4, 1), dtype=float)
    u_zero = -np.sin(np.pi * x_zero)

    # stack them in the same order as X_u_train was stacked:
    u_train = np.vstack( (u_upper, u_lower, u_zero) )

    # match indices with X_u_train
    u_train = u_train[index, :]
    # Model instantiation
    model = BurgersNN()
    model.apply(init_weights)
    # Training
    X_u_train = torch.from_numpy(X_u_train).requires_grad_(True).float()
    X_f_train = torch.from_numpy(X_f_train).requires_grad_(True).float()
    u_train = torch.from_numpy(u_train).requires_grad_(True).float()

    model.train()
    train(model, X_u_train, X_f_train, u_train)
    # save the model
    torch.save(model.state_dict(), 'Burgers_Equation/models/model_LBFGS_shuffle_normal.pt')


 iteration: 1  loss: 0.2293640524148941
 iteration: 2  loss: 0.21785911917686462
 iteration: 3  loss: 0.20473024249076843
 iteration: 4  loss: 0.19932690262794495
 iteration: 5  loss: 0.1981697678565979
 iteration: 6  loss: 0.1948550045490265
 iteration: 7  loss: 0.1793188750743866
 iteration: 8  loss: 0.19664618372917175
 iteration: 9  loss: 0.1724817156791687
 iteration: 10  loss: 0.16531942784786224
 iteration: 11  loss: 0.1623375117778778
 iteration: 12  loss: 0.15767674148082733
 iteration: 13  loss: 0.15157198905944824
 iteration: 14  loss: 0.15005958080291748
 iteration: 15  loss: 0.1489448994398117
 iteration: 16  loss: 0.14304740726947784
 iteration: 17  loss: 0.13657788932323456
 iteration: 18  loss: 2.2932674884796143
 iteration: 19  loss: 0.13524307310581207
 iteration: 20  loss: 0.132037952542305
 iteration: 21  loss: 0.12714479863643646
 iteration: 22  loss: 0.12551532685756683
 iteration: 23  loss: 0.12286354601383209
 iteration: 24  loss: 0.12094718962907791
 iteration:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
from main import BurgersNN

# load model
model = BurgersNN()
model.load_state_dict(torch.load('Burgers_Equation/models/model_LBFGS_shuffle_normal.pt'))
x = torch.linspace(-1, 1, 200)
t = torch.linspace( 0, 1, 100)

# x & t grids:
X, T = torch.meshgrid(x, t)

# x & t columns:
xcol = X.reshape(-1, 1)
tcol = T.reshape(-1, 1)
input = torch.cat((xcol, tcol), 1)
# one large column:
usol = model(input)

# reshape solution:
U = usol.reshape(x.numel(), t.numel())

# transform to numpy:
xnp = x.numpy()
tnp = t.numpy()
Unp = U.detach().numpy()

# plot:
fig = plt.figure(figsize=(9, 4.5))
ax = fig.add_subplot(111)

h = ax.imshow(Unp,
                interpolation='nearest',
                cmap='rainbow',
                extent=[tnp.min(), tnp.max(), xnp.min(), xnp.max()],
                origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.10)
cbar = fig.colorbar(h, cax=cax)
cbar.ax.tick_params(labelsize=10)
plt.show()